<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# ***Título do seu projeto***
---

*Siga esse template para entrega do seu projeto. O código deste notebook precisa ser 100% executável.*

In [1]:
#@title Identificação do Grupo

#@markdown Turma **7N**

#@markdown Integrantes do Grupo (*informe \<TIA\>,\<nome\>*)
Aluno1 = '31944043, Giancarlo Ponte Gamberi' #@param {type:"string"} 
Aluno2 = ' 31938698, Rone Tadeu Filho' #@param {type:"string"}
Aluno3 = 'None' #@param {type:"string"}
Aluno4 = 'None' #@param {type:"string"}
Aluno5 = 'None' #@param {type:"string"}

# Apresentação do Trabalho

*Apresente resumidamente o trabalho justificando a escolha dos dados.*

# Referencial Teórico

*Resuma aqui referências sobre o uso da regressão linear $times$ regressão logística na classificação de dados.* 

# Base de Dados 

*Descreva aqui os dados utilizados, discuta eventuais transformações e/ou seleções dos dados e preparações nos dados.* 





Selecionamos uma base de dados que contem o preco de stock em tempo real da empresa IBM das datas: 06-04-2020 - 25-03-2022,

Um dos motivos de ter sido escolhida para este trabalho, eh o fato de ser uma base quase inteiramente numerica, como unico desafio sendo elaborar uma coluna classificativa, que no caso sera se uma acao fechara em alta ou nao, 
('superavit': 0,1)

In [31]:
# importacoes
import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import warnings
import os
import seaborn as sns
import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")

In [30]:
# Codigo utilizado inicialmente para carregar o arquivo do kaggle

# *Verificar se o arquivo kaggle.json esta presente antes de prosseguir para execucao, caso nao, gerar um token de API no keggle e carregar o arquivo

# tutorial:
# https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/

# #importacao dos dados

# ! pip install kaggle
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download -d bhanuprasanna527/stock-market-prediction
# ! unzip /content/stock-market-prediction.zip


In [35]:
#abrindo CSV
IBM = pd.read_csv('https://raw.githubusercontent.com/REXDES/N1-IA-Projeto/main/2%20Year%20IBM%20Stock%20Data.csv')

#removendo nulos
IBM = IBM.dropna()

# Adicionando coluna superavit, que classifica binariamente se uma acao subiu ou nao, em relacao ao preco de abertura e fechamento do dia
IBM['superavit'] = (IBM['close']>IBM['open'])/1

IBM.head()


,time,open,high,low,close,volume,superavit
0,3/25/2022 19:19,131.300,131.30,131.300,131.30,718,0.0
1,3/25/2022 19:18,131.300,131.30,131.300,131.30,282,0.0
2,3/25/2022 17:50,131.300,131.30,131.300,131.30,100,0.0
3,3/25/2022 16:59,131.340,131.35,131.340,131.35,1427,1.0
4,3/25/2022 16:42,131.326,131.35,131.326,131.35,1326,1.0


# Modelo de Regressão Linear

*Descreva aqui as variáveis dependentes e independentes empregadas e eventuais transformações nas variáveis.*


In [36]:
# separando modelo para treinamento/teste
X_train, X_test, y_train, y_test = train_test_split(IBM.drop(columns=['superavit']), IBM.superavit, test_size=0.3, stratify=IBM.superavit, random_state=1984)

# Base de teste para ser carregada no Modelo
IBMLin = pd.concat([X_train, y_train],axis=1)

print(IBMLin)

                    time        open        high         low       close  \
20191    1/19/2022 14:50  130.674945  130.704581  130.665067  130.694703   
188163    7/9/2020 11:02  101.269504  101.321998  101.103274  101.164517   
47863   10/20/2021 12:37  132.144763  132.163399  132.126128  132.126128   
134821  12/31/2020 15:30  112.783396  112.783666  112.765420  112.765420   
188931    7/7/2020 12:33  103.714846  103.771714  103.704434  103.736718   
...                  ...         ...         ...         ...         ...   
208301    5/1/2020 14:44  105.135910  105.135910  105.088429  105.088429   
10258    2/18/2022 14:26  124.750000  124.830000  124.660000  124.660000   
145538  11/23/2020 13:45  107.678123  107.709581  107.651158  107.660147   
149532  11/10/2020 11:46  105.835551  105.855145  105.763645  105.763645   
91421    5/24/2021 11:42  133.409763  133.465028  133.409763  133.460423   

        volume  superavit  
20191     5472        1.0  
188163   15082        0.0  
478

In [37]:
# Modelo, formula utilizada sendo 'superavit <- open + close'
model = sm.ols(formula="superavit ~ open + close ", data=IBMLin)

result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              superavit   R-squared:                       0.335
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                 3.826e+04
Date:                Wed, 06 Apr 2022   Prob (F-statistic):               0.00
Time:                        22:45:58   Log-Likelihood:                -78065.
No. Observations:              151818   AIC:                         1.561e+05
Df Residuals:                  151815   BIC:                         1.562e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4086      0.011     36.424      0.0

## Análise do Modelo Linear

Como os dados de predicao retornam valores quebrados fload, vamos considerar predicoes maiores que 0.5 como sendo a classe 1, enquanto predicoes menores ou iguais que 0.5 como sendo da classe 0

In [38]:
y_pred = result.predict(X_test)

#retornando os dados float para binariedade
y_pred = (y_pred > 0.5)/1

#visualizacao dos dados
print (y_pred.head())
print (y_test.head())

# acuracidade
print (sum(y_pred==y_test)/len(y_pred))

80166     1.0
85089     1.0
185902    0.0
159588    1.0
67086     0.0
dtype: float64
80166     1.0
85089     1.0
185902    0.0
159588    1.0
67086     0.0
Name: superavit, dtype: float64
0.8901867363405825


## Discussão dos Resultados do Modelo Linear

*Discuta aqui os resultados obtidos no modelo.*


# Modelo Logístico

*Descreva aqui as variáveis dependentes e independentes empregadas e eventuais transformações nas variáveis.*


In [39]:
IBMLog = IBM[['superavit','open','close']]

#separando modelo para treinamento/teste
X_train, X_test, y_train, y_test = train_test_split(IBMLog.drop(columns=['superavit']), IBMLog.superavit, test_size=0.3, stratify=IBMLog.superavit, random_state=1984)

logreg = LogisticRegression()
logreg.fit(X_train,y_train)

LogisticRegression()

## Análise do Modelo Logístico

In [40]:
y_pred=logreg.predict(X_test)

print (y_pred)
print (y_test)

print (sum(y_pred==y_test)/len(y_pred))

[1. 1. 0. ... 0. 1. 1.]
80166     1.0
85089     1.0
185902    0.0
159588    1.0
67086     0.0
         ... 
20686     0.0
20435     1.0
203216    0.0
165705    1.0
107179    1.0
Name: superavit, Length: 65065, dtype: float64
0.9745946361330977


## Discussão dos Resultados do Modelo Logístico

*Discuta aqui os resultados obtidos no modelo.*


In [ ]:
# seu código

# Conclusão 

*Apresente a conclusão do seu estudo comparando ainda os resultados obtidos com o referencial teórico apresentado.*



# Referências 

*Indique as referências empregadas, incluindo as fontes de dados.*

Fonte de dados:
https://www.kaggle.com/datasets/bhanuprasanna527/stock-market-prediction

Notebooks referencia e referencial teorico:
https://github.com/Rogerio-mack/IA_2022_1S/blob/main/README.md

Tutorial utilizado incialmente para carregar o arquivo csv do kaggle:
https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/

---

In [ ]:
#@title Avaliação
Completo = 10 #@param {type:"slider", min:0, max:10, step:1}
#@markdown Projeto cumpre todos os itens pedidos. 
Relevancia = 9 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As seleções de dados e eventos para análise são relevantes e justificados. 
Tecnicas = 6 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As técnicas de empregadas são adequadas e corretamente aplicadas.
Apresentacao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown A apresentação dos resultados é clara e objetiva.
Analise = 8 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As premissas de análise se justificam e a analise é correta. 
Conclusao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As conclusões são justificadas e relevantes
Bonus = 0.5 #@param {type:"slider", min:0, max:1, step:0.5}
#@markdown A critério do professor por inovações na abordagem e no uso de técnicas de Análise de Dados








In [ ]:
#@markdown ### Nota Final
nota = Completo + Relevancia + Tecnicas + Apresentacao + Analise + Conclusao 

nota = nota / 6 + Bonus

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
alunos

Nota final do trabalho 8.3



,tia,nome,nota
0,1115665,ADRIANA FUJITA,8.3
1,1115677,DANIEL HENRIQUE,8.3
